<a href="https://colab.research.google.com/github/PremGorecki/NeuralNetwork/blob/main/03_keras/04_callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Update: Należy użyć biblioteki tensorflow w wersji 2.0
!pip install -q tensorflow==2.0

In [ ]:
# %tensorflow_version 2.x  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

np.set_printoptions(precision=12, suppress=True, linewidth=120)
print(tf.__version__)

Załadowanie i przygotowanie danych

In [ ]:
(X_train, y_train), (X_test, y_test) = load_data()

print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

In [ ]:
print(X_train[0])

In [ ]:
print(f'X_train[0] shape: {X_train[0].shape}')

In [8]:
X_train = X_train / 255.
X_test = X_test / 255.

In [ ]:
#Eksploracja danych

plt.imshow(X_train[0], cmap='gray_r')
plt.axis('off')

In [ ]:
plt.figure(figsize=(13, 13))
for i in range(1, 11):
    plt.subplot(1, 10, i)
    plt.axis('off')
    plt.imshow(X_train[i-1], cmap='gray_r')
    plt.title(y_train[i-1], color='black', fontsize=16)
plt.show()

Budowa sieci neuronowej

In [ ]:
def build_model():
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(units=128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(units=10, activation='softmax'))

    model.compile(optimizers='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

model = build_model()
model.summary()

Trenowanie modelu

In [13]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 6s 123us/sample - loss: 0.3155 - accuracy: 0.9078 - val_loss: 0.1651 - val_accuracy: 0.9517
Epoch 2/10
48000/48000 [==============================] - 5s 95us/sample - loss: 0.1641 - accuracy: 0.9525 - val_loss: 0.1249 - val_accuracy: 0.9655
Epoch 3/10
48000/48000 [==============================] - 4s 91us/sample - loss: 0.1312 - accuracy: 0.9627 - val_loss: 0.1121 - val_accuracy: 0.9684
Epoch 4/10
48000/48000 [==============================] - 4s 86us/sample - loss: 0.1124 - accuracy: 0.9680 - val_loss: 0.1179 - val_accuracy: 0.9697
Epoch 5/10
48000/48000 [==============================] - 4s 93us/sample - loss: 0.1023 - accuracy: 0.9707 - val_loss: 0.1065 - val_accuracy: 0.9736
Epoch 6/10
48000/48000 [==============================] - 4s 89us/sample - loss: 0.0927 - accuracy: 0.9740 - val_loss: 0.1110 - val_accuracy: 0.9735
Epoch 7/10
48000/48000 [==============================]

ModelCheckpoint - Zapisywanie wag po każdej poprawie modelu
Za każdym razem gdy monitorowana metryka ulega poprawie wagi są zapisywane do pliku.
Wykorzystamy klasę ModelCheckpoint do obserwowania metryki accuracy na zbiorze walidacyjnym.

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint

!mkdir model
filepath ='model/weights-{epoch:02d}-{val_accuracy:.4f}.hdf5'

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
#polecenie bash'owe do wyświetlenia katalogu w którym się znajdujemy
!ls

In [ ]:
model = build_model()
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32, callbacks=[checkpoint])

ModelCheckpoint - Zapisanie najlepszego modelu

In [ ]:
filepath = 'best_model_weights.hdf5'

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

model = build_model()
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=32, callbacks=[checkpoint])